In [5]:
import cv2
import csv
import os
import math
import numpy as np
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
# VIDEO_PATH = r"F:\New folder\video_20260111_144158 - Trim - Trim.mp4"
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"

SAVE_DIR = "detections"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "final_human_coordinates.csv")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== CAMERA CALIBRATION =====================
CAMERA_MATRIX = np.array([
    [1176.58,    0.0,  999.32],
    [   0.0, 1176.61,  522.03],
    [   0.0,    0.0,     1.0]
], dtype=np.float64)

DIST_COEFFS = np.array([
    0.25010672,
   -0.53448585,
   -0.01314542,
    0.01928691,
    0.4317226
], dtype=np.float64)

fx = CAMERA_MATRIX[0, 0]
fy = CAMERA_MATRIX[1, 1]
cx0 = CAMERA_MATRIX[0, 2]
cy0 = CAMERA_MATRIX[1, 2]

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.5

BOTTOM_RATIO = 0.5          # bottom 40%
CENTER_LEFT_RATIO = 0.3
CENTER_RIGHT_RATIO = 0.7

CAM_PITCH = math.radians(35)   # 35° downward from horizontal

# ===================== HARDCODED DRONE STATE =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0   # meters AGL

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "person_id", "human_lat", "human_lon"])

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(1)

logged_person_ids = set()

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    # Undistort frame
    frame = cv2.undistort(frame, CAMERA_MATRIX, DIST_COEFFS)

    h, w = frame.shape[:2]
    bottom_limit = int(BOTTOM_RATIO * h)

    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is None:
        cv2.imshow("Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        continue

    for box in results.boxes:

        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        if box.id is None:
            continue

        person_id = int(box.id[0])

        if person_id in logged_person_ids:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        u = (x1 + x2) / 2
        v = (y1 + y2) / 2

        # -------- Bottom 40% filter --------
        if v < bottom_limit:
            continue

        # -------- Clamp horizontal offset --------
        center_left_px = CENTER_LEFT_RATIO * w
        center_right_px = CENTER_RIGHT_RATIO * w

        if center_left_px < u < center_right_px:
            u_used = cx0
        elif u <= center_left_px:
            u_used = center_left_px
        else:
            u_used = center_right_px

        v_used = bottom_limit

        # -------- Pixel → angles --------
        theta_x = math.atan((u_used - cx0) / fx)
        theta_y = math.atan((v_used - cy0) / fy)

        # -------- Ground intersection --------
        ground_angle = CAM_PITCH + theta_y
        if ground_angle <= 0:
            continue

        forward_dist = DRONE_ALT / math.tan(ground_angle)
        lateral_dist = forward_dist * math.tan(theta_x)

        # -------- Global GPS --------
        R_earth = 6378137.0
        delta_lat = (forward_dist / R_earth) * (180 / math.pi)
        delta_lon = (lateral_dist / (R_earth * math.cos(math.radians(DRONE_LAT)))) * (180 / math.pi)

        human_lat = DRONE_LAT + delta_lat
        human_lon = DRONE_LON + delta_lon

        # -------- Save image --------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(
            IMG_DIR, f"human_id{person_id}_{timestamp}.jpg"
        )

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"ID:{person_id}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

        cv2.imwrite(img_path, frame)

        # -------- Save CSV --------
        with open(CSV_PATH, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                timestamp,
                person_id,
                round(human_lat, 7),
                round(human_lon, 7)
            ])

        logged_person_ids.add(person_id)
        print(f"[SAVED] ID {person_id} @ ({human_lat:.7f}, {human_lon:.7f})")

    # -------- Visual guides --------
    cv2.line(frame, (0, bottom_limit), (w, bottom_limit), (0, 0, 255), 2)
    cv2.line(frame, (int(CENTER_LEFT_RATIO * w), 0), (int(CENTER_LEFT_RATIO * w), h), (255, 255, 0), 1)
    cv2.line(frame, (int(CENTER_RIGHT_RATIO * w), 0), (int(CENTER_RIGHT_RATIO * w), h), (255, 255, 0), 1)

    cv2.imshow("Human Detection (Near-Only)", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


[SAVED] ID 1 @ (25.2624509, 82.9840395)
[SAVED] ID 4 @ (25.2624509, 82.9838624)
[SAVED] ID 6 @ (25.2624509, 82.9840395)


In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import math
import time
import csv
from geopy.distance import geodesic
import os
from collections import defaultdict
from collections import deque



In [ ]:
# Drone GPS 
drone_lon = 82.9840395
drone_lat = 25.2621092
drone_alt = 15

In [ ]:
# Drone IMU
imu_roll = 0.0
imu_pitch = 0.0
imu_yaw = 90

In [ ]:
# Gimbal oriantation
gimbal_roll = 0
gimbal_pitch = -35
gimbal_yaw =0

In [ ]:
# Convert camera details into matrix form (roll, pitch and yaw are in radians)
def R_x(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [1, 0, 0],
        [0, ca, -sa],
        [0, sa,  ca]
    ])

def R_y(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [ ca, 0, sa],
        [  0, 1,  0],
        [-sa, 0, ca]
    ])

def R_z(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [ca, -sa, 0],
        [sa,  ca, 0],
        [ 0,   0, 1]
    ])

def get_camera_to_world_rotation(imu_roll, imu_pitch, imu_yaw, gimbal_pitch):
    
    # All angles in RADIANS
    # imu_     : drone body attitude (from MAVLink ATTITUDE)
    # gimbal_  : camera gimbal angles
    
    # Camera -> Gimbal
    # Camera optical axis (Z forward) →  Drone body forward (X)
    R_cam_gimbal = np.array([
        [ 0,  1,  0],
        [ 1,  0,  0],
        [ 0,  0, -1]
    ])
    # Gimbal -> Body 
    R_gimbal_body = (
        R_y(gimbal_pitch) 
    )

    # Body -> World 
    R_body_world = (
        R_z(imu_yaw) @
        R_y(imu_pitch) @
        R_x(imu_roll)
    )

    # Final rotation matrix
    return R_body_world @ R_gimbal_body @ R_cam_gimbal


In [ ]:
CALIB_WIDTH = 1920
CALIB_HEIGHT = 1080

def scale_camera_matrix(CAMERA_MATRIX, w, h):
    sx = w / CALIB_WIDTH
    sy = h / CALIB_HEIGHT

    K_scaled = CAMERA_MATRIX.copy()
    K_scaled[0, 0] *= sx   # fx
    K_scaled[1, 1] *= sy   # fy
    K_scaled[0, 2] *= sx   # cx
    K_scaled[1, 2] *= sy   # cy

    return K_scaled



In [ ]:
# callibrated
CAMERA_MATRIX = np.array([
    [1176.58,    0.0,  999.32],
    [   0.0,  1176.61, 522.03],
    [   0.0,     0.0,    1.0]
])

DIST_COEFFS = np.array([
    0.25010672,
   -0.53448585,
   -0.01314542,
    0.01928691,
    0.4317226
])

def get_geotag(
    bbox,
    frame,
    drone_lat, drone_lon, drone_alt,
    imu_roll, imu_pitch, imu_yaw,
    gimbal_pitch
):
    x1, y1, x2, y2 = bbox
    u = (x1 + x2) * 0.5
    v = y2 
    
    K = scale_camera_matrix(CAMERA_MATRIX, frame.shape[1], frame.shape[0])
    # Undistort the POINT (u, v)
    pts = np.array([[[u, v]]], dtype=np.float32)

    pts_undist = cv2.undistortPoints(
        pts,
        CAMERA_MATRIX,
        DIST_COEFFS,
        P=K        # returns pixel coordinates
    )

    u_undist, v_undist = pts_undist[0, 0]
    


    fx = K[0, 0]
    fy = K[1, 1]
    cx = K[0, 2]
    cy = K[1, 2]


    # Pixel → camera ray
    x_cam = (u_undist - cx) / fx
    y_cam = -(v_undist - cy) / fy
    z_cam = -1.0

    ray_cam = np.array([x_cam, y_cam, z_cam])
    ray_cam /= np.linalg.norm(ray_cam)

    # Rotations
    roll  = math.radians(imu_roll)
    pitch = math.radians(imu_pitch + gimbal_pitch)
    yaw   = math.radians(imu_yaw)

    Rz = np.array([
        [ math.cos(yaw), -math.sin(yaw), 0],
        [ math.sin(yaw),  math.cos(yaw), 0],
        [ 0,              0,             1]
    ])

    Ry = np.array([
        [ math.cos(pitch), 0, math.sin(pitch)],
        [ 0,               1, 0],
        [-math.sin(pitch), 0, math.cos(pitch)]
    ])

    Rx = np.array([
        [1, 0,              0],
        [0, math.cos(roll), -math.sin(roll)],
        [0, math.sin(roll),  math.cos(roll)]
    ])

    R = Rz @ Ry @ Rx
    ray_world = R @ ray_cam

    if ray_world[2] >= -1e-6:
        return None

    t = drone_alt /(-ray_world[2])
    east  = t * ray_world[0]
    north = t * ray_world[1]

    R_earth = 6378137.0

    dlat = (north / R_earth) * (180 / math.pi)
    dlon = (east  / (R_earth * math.cos(math.radians(drone_lat)))) * (180 / math.pi)


    return drone_lat + dlat, drone_lon + dlon


In [ ]:
def drop(lat, lon):
    print(f"🚑 DROPPING PARCEL at {lat}, {lon}")



def surveillance():
    print("🛰️ SURVEILLANCE MODE")
    


In [ ]:
class KalmanGPS:
    def __init__(self, lat, lon):
        self.x = np.array([[lat], [lon]])
        self.P = np.eye(2) * 1e-4
        self.Q = np.eye(2) * 1e-6
        self.R = np.eye(2) * 1e-5

    def update(self, lat, lon):
        z = np.array([[lat], [lon]])
        self.P = self.P + self.Q
        K = self.P @ np.linalg.inv(self.P + self.R)
        self.x = self.x + K @ (z - self.x)
        self.P = (np.eye(2) - K) @ self.P
        return float(self.x[0]), float(self.x[1])


In [ ]:
PERSON_CLASS_ID = 0
CONF_TH = 0.50
FRAME_CONFIRM = 10         
SERVE_RADIUS = 1     
SAME_PERSON_RADIUS = 1
AVG_WINDOW = 10 
      

#rtsp_url = "rtsp://192.168.144.25:8554/main.264"
# rtsp_url = r"F:\New folder\video_20260111_144158 - Trim.mp4"
cap = cv2.VideoCapture(1)

# cap = cv2.VideoCapture(r"E:\NIDAR\test\final\WhatsApp Video 2025-12-17 at 12.29.52.mp4")
model = YOLO(r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt")

served_targets = []            
candidate_tracks = {}          
next_candidate_id = 1
csv_buffer = []



In [ ]:
def is_served(lat, lon):
    return any(
        geodesic((lat, lon), s).meters < SERVE_RADIUS
        for s in served_targets
    )

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=CONF_TH, verbose=False)[0]

    for box in results.boxes:
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        # ---- Bounding box from YOLO (THIS IS ALL YOU NEED) ----
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        gps = get_geotag(
            bbox=(x1, y1, x2, y2),
            frame = frame,
            drone_lat=drone_lat,
            drone_lon=drone_lon,
            drone_alt=drone_alt,
            imu_roll=imu_roll,
            imu_pitch=imu_pitch,
            imu_yaw=imu_yaw,
            gimbal_pitch=gimbal_pitch
        )

        # gps = get_geotag_precise(
        #     bbox=(x1, y1, x2, y2),
        #     frame = frame,
        #     drone_lat=drone_lat,
        #     drone_lon=drone_lon,
        #     drone_alt=drone_alt,
        #     yaw_deg = imu_yaw,          # heading of drone
        #     gimbal_pitch_deg  = gimbal_pitch     # NEGATIVE (e.g. -35)
        # )

#         gps = get_geotag_precise(
#             drone_lat = drone_lat,
#             drone_lon = drone_lon,
#             drone_alt = drone_alt,        # meters AGL
#             yaw_deg = imu_yaw,          # ABSOLUTE yaw (0 = North, CW positive)
#             gimbal_pitch_deg = gimbal_pitch, # negative down (e.g. -35)
#             bbox = (x1, y1, x2, y2),
#             img_w = frame.shape[1],
#             img_h = frame.shape[0],
#             hfov_deg = 81.4
# )

        if gps is None:
            continue

        lat, lon = gps

        if is_served(lat, lon):
            continue

        # ---- TRACK MATCHING ----
        matched_id = None
        for cid, track in candidate_tracks.items():
            if track["confirmed"]:
                continue
            d = geodesic((lat, lon), (track["lat"], track["lon"])).meters
            if d < SAME_PERSON_RADIUS:
                matched_id = cid
                break

        # ---- NEW PERSON ----
        if matched_id is None:
            candidate_tracks[next_candidate_id] = {
                "lat": lat,
                "lon": lon,
                "count": 1,
                "lat_buf": deque([lat], maxlen=AVG_WINDOW),
                "lon_buf": deque([lon], maxlen=AVG_WINDOW),
                "confirmed": False
            }
            print(f"[NEW] Person ID {next_candidate_id}")
            matched_id = next_candidate_id
            next_candidate_id += 1

        # ---- EXISTING PERSON ----
        else:
            track = candidate_tracks[matched_id]
            track["count"] += 1

            track["lat"] = lat
            track["lon"] = lon
            track["lat_buf"].append(lat)
            track["lon_buf"].append(lon)

            print(f"[TRACK] ID {matched_id} count={track['count']}")

            # ---- CONFIRM PERSON ----
            if track["count"] >= FRAME_CONFIRM and not track["confirmed"]:
                avg_lat = float(np.median(track["lat_buf"]))
                avg_lon = float(np.median(track["lon_buf"]))

                print(
                    f"🎯 DROP TARGET ID {matched_id}: "
                    f"{avg_lat:.6f}, {avg_lon:.6f}"
                )

                served_targets.append((avg_lat, avg_lon))
                track["confirmed"] = True

                csv_buffer.append([
                    matched_id,
                    avg_lat,
                    avg_lon,
                    time.strftime("%Y-%m-%d %H:%M:%S")
                ])


# ================= SAVE CSV =================
with open("geotagged_person_locations.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["person_id", "latitude", "longitude", "timestamp"])
    writer.writerows(csv_buffer)

cap.release()
print("🛑 SYSTEM STOPPED")

In [ ]:
import cv2
import csv
import os
import math
import numpy as np
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\dowloads\PixVerse_V5.5_Image_Text_360P_I_want_a_video_i.mp4"

SAVE_DIR = "detections"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "final_human_coordinates.csv")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== CAMERA CALIBRATION =====================
CAMERA_MATRIX = np.array([
    [1176.58, 0.0, 999.32],
    [0.0, 1176.61, 522.03],
    [0.0, 0.0, 1.0]
], dtype=np.float64)

DIST_COEFFS = np.array([
    0.25010672,
   -0.53448585,
   -0.01314542,
    0.01928691,
    0.4317226
], dtype=np.float64)

fx = CAMERA_MATRIX[0, 0]
fy = CAMERA_MATRIX[1, 1]
cx0 = CAMERA_MATRIX[0, 2]
cy0 = CAMERA_MATRIX[1, 2]

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.60        # 🔥 STRICT CONFIDENCE THRESHOLD

BOTTOM_RATIO = 0.5
CENTER_LEFT_RATIO = 0.3
CENTER_RIGHT_RATIO = 0.7

CAM_PITCH = math.radians(35)
FRAME_CONFIRM = 7
DUPLICATE_RADIUS_M = 1.5
CELL_SIZE_M = 1.0

# ===================== DRONE STATE =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "human_id", "latitude", "longitude"])

# ===================== HELPERS =====================
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6378137.0
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(math.radians(lat1)) *
         math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) ** 2)
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# ===================== MODEL & CAMERA =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
humans = []                 # [{"id": int, "lat": float, "lon": float}]
saved_ids = set()           # IDs already written to CSV
candidate_counter = {}      # {(cell_x, cell_y): count}
next_human_id = 1

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.undistort(frame, CAMERA_MATRIX, DIST_COEFFS)
    h, w = frame.shape[:2]
    bottom_limit = int(BOTTOM_RATIO * h)

    results = model(frame, conf=CONF_TH, verbose=False)[0]
    seen_cells = set()

    if results.boxes is not None:
        for box in results.boxes:

            # ---------- CLASS FILTER ----------
            if int(box.cls[0]) != PERSON_CLASS_ID:
                continue

            # ---------- CONFIDENCE FILTER (FINAL GUARD) ----------
            conf = float(box.conf[0])
            if conf < 0.65:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            u = (x1 + x2) / 2
            v = (y1 + y2) / 2

            # ---------- BOTTOM-FRAME FILTER ----------
            if v < bottom_limit:
                continue

            # ---------- CLAMP HORIZONTAL OFFSET ----------
            cl = CENTER_LEFT_RATIO * w
            cr = CENTER_RIGHT_RATIO * w

            if cl < u < cr:
                u_used = cx0
            elif u <= cl:
                u_used = cl
            else:
                u_used = cr

            v_used = bottom_limit

            # ---------- PIXEL → ANGLES ----------
            theta_x = math.atan((u_used - cx0) / fx)
            theta_y = math.atan((v_used - cy0) / fy)

            ground_angle = CAM_PITCH + theta_y
            if ground_angle <= 0:
                continue

            forward_dist = DRONE_ALT / math.tan(ground_angle)
            lateral_dist = forward_dist * math.tan(theta_x)

            R = 6378137.0
            dlat = (forward_dist / R) * (180 / math.pi)
            dlon = (lateral_dist / (R * math.cos(math.radians(DRONE_LAT)))) * (180 / math.pi)

            human_lat = DRONE_LAT + dlat
            human_lon = DRONE_LON + dlon

            cell = (
                int(forward_dist / CELL_SIZE_M),
                int(lateral_dist / CELL_SIZE_M)
            )

            seen_cells.add(cell)
            candidate_counter[cell] = candidate_counter.get(cell, 0) + 1

            # ---------- 7-FRAME CONFIRMATION ----------
            if candidate_counter[cell] == FRAME_CONFIRM:

                assigned_id = None
                for hmn in humans:
                    if haversine_m(hmn["lat"], hmn["lon"], human_lat, human_lon) < DUPLICATE_RADIUS_M:
                        assigned_id = hmn["id"]
                        break

                if assigned_id is None:
                    assigned_id = next_human_id
                    humans.append({
                        "id": assigned_id,
                        "lat": human_lat,
                        "lon": human_lon
                    })
                    next_human_id += 1

                # ---------- SAVE ONLY ONCE PER ID ----------
                if assigned_id not in saved_ids:
                    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

                    img_path = os.path.join(IMG_DIR, f"human_{assigned_id}_{ts}.jpg")
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.imwrite(img_path, frame)

                    with open(CSV_PATH, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            ts,
                            assigned_id,
                            round(human_lat, 7),
                            round(human_lon, 7)
                        ])

                    saved_ids.add(assigned_id)
                    print(f"✅ SAVED HUMAN ID {assigned_id} (conf={conf:.2f})")

    candidate_counter = {k: v for k, v in candidate_counter.items() if k in seen_cells}

    cv2.line(frame, (0, bottom_limit), (w, bottom_limit), (0, 0, 255), 2)
    cv2.imshow("Human Detection (CONF ≥ 0.65)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


✅ SAVED HUMAN ID 1 (conf=0.66)
✅ SAVED HUMAN ID 2 (conf=0.72)


In [8]:
import cv2
import csv
import os
import math
import numpy as np
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH =  r"F:\dowloads\PixVerse_V5.5_Image_Text_360P_I_want_a_video_i.mp4"

SAVE_DIR = "detections"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "final_human_coordinates.csv")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== CAMERA CALIBRATION =====================
CAMERA_MATRIX = np.array([
    [1176.58, 0.0, 999.32],
    [0.0, 1176.61, 522.03],
    [0.0, 0.0, 1.0]
], dtype=np.float64)

DIST_COEFFS = np.array([
    0.25010672,
   -0.53448585,
   -0.01314542,
    0.01928691,
    0.4317226
], dtype=np.float64)

fx = CAMERA_MATRIX[0, 0]
fy = CAMERA_MATRIX[1, 1]
cx0 = CAMERA_MATRIX[0, 2]
cy0 = CAMERA_MATRIX[1, 2]

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.60

BOTTOM_RATIO = 0.5
CENTER_LEFT_RATIO = 0.3
CENTER_RIGHT_RATIO = 0.7

CAM_PITCH = math.radians(35)
FRAME_CONFIRM = 4
CELL_SIZE_M = 1.0

# ===================== DRONE STATE =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "human_id", "latitude", "longitude"])

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
saved_ids = set()
candidate_counter = {}
cell_to_id = {}
next_human_id = 1

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.undistort(frame, CAMERA_MATRIX, DIST_COEFFS)
    h, w = frame.shape[:2]
    bottom_limit = int(BOTTOM_RATIO * h)

    results = model(frame, conf=CONF_TH, verbose=False)[0]
    seen_cells = set()

    if results.boxes is not None:
        for box in results.boxes:

            # -------- CLASS --------
            if int(box.cls[0]) != PERSON_CLASS_ID:
                continue

            # -------- CONFIDENCE --------
            conf = float(box.conf[0])
            if conf < CONF_TH:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            u = (x1 + x2) / 2
            v = (y1 + y2) / 2

            # -------- BOTTOM FILTER --------
            if v < bottom_limit:
                continue

            # -------- CLAMP --------
            cl = CENTER_LEFT_RATIO * w
            cr = CENTER_RIGHT_RATIO * w

            if cl < u < cr:
                u_used = cx0
            elif u <= cl:
                u_used = cl
            else:
                u_used = cr

            v_used = bottom_limit

            # -------- ANGLES --------
            theta_x = math.atan((u_used - cx0) / fx)
            theta_y = math.atan((v_used - cy0) / fy)

            ground_angle = CAM_PITCH + theta_y
            if ground_angle <= 0:
                continue

            forward_dist = DRONE_ALT / math.tan(ground_angle)
            lateral_dist = forward_dist * math.tan(theta_x)

            R = 6378137.0
            dlat = (forward_dist / R) * (180 / math.pi)
            dlon = (lateral_dist / (R * math.cos(math.radians(DRONE_LAT)))) * (180 / math.pi)

            human_lat = DRONE_LAT + dlat
            human_lon = DRONE_LON + dlon

            cell = (
                int(forward_dist / CELL_SIZE_M),
                int(lateral_dist / CELL_SIZE_M)
            )

            seen_cells.add(cell)
            candidate_counter[cell] = candidate_counter.get(cell, 0) + 1

            # -------- CONFIRM AFTER 7 FRAMES --------
            if candidate_counter[cell] == FRAME_CONFIRM:

                if cell not in cell_to_id:
                    cell_to_id[cell] = next_human_id
                    next_human_id += 1

                assigned_id = cell_to_id[cell]

                if assigned_id not in saved_ids:
                    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

                    # ✅ DRAW BOUNDING BOX ON IMAGE
                    save_img = frame.copy()
                    cv2.rectangle(save_img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(
                        save_img,
                        f"ID {assigned_id}",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 255, 0),
                        2
                    )

                    img_path = os.path.join(
                        IMG_DIR, f"human_{assigned_id}_{ts}.jpg"
                    )
                    cv2.imwrite(img_path, save_img)

                    with open(CSV_PATH, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            ts,
                            assigned_id,
                            round(human_lat, 7),
                            round(human_lon, 7)
                        ])

                    saved_ids.add(assigned_id)
                    print(f"✅ SAVED HUMAN ID {assigned_id} (conf={conf:.2f})")

    # cleanup
    candidate_counter = {
        k: v for k, v in candidate_counter.items() if k in seen_cells
    }

    cv2.line(frame, (0, bottom_limit), (w, bottom_limit), (0, 0, 255), 2)
    cv2.imshow("Human Detection (BBox Saved)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


✅ SAVED HUMAN ID 1 (conf=0.71)
✅ SAVED HUMAN ID 2 (conf=0.67)
✅ SAVED HUMAN ID 3 (conf=0.60)


In [12]:
import cv2
import os
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\New folder\video_20260111_144158 - Trim - Trim.mp4"

SAVE_DIR = "detected_humans"
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.65   # confidence threshold

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
saved_track_ids = set()   # save each person only once

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO with tracking
    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is None:
        cv2.imshow("Human Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        continue

    for box in results.boxes:

        # -------- CLASS FILTER --------
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        # -------- TRACK ID CHECK --------
        if box.id is None:
            continue

        track_id = int(box.id[0])

        # Already saved → skip
        if track_id in saved_track_ids:
            continue

        # -------- CONFIDENCE FILTER --------
        conf = float(box.conf[0])
        if conf < CONF_TH:
            continue

        # -------- DRAW BOUNDING BOX --------
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        save_frame = frame.copy()

        cv2.rectangle(save_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(
            save_frame,
            f"Human ID {track_id}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2
        )

        # -------- SAVE FRAME --------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(
            SAVE_DIR, f"human_{track_id}_{timestamp}.jpg"
        )

        cv2.imwrite(img_path, save_frame)
        saved_track_ids.add(track_id)

        print(f"✅ SAVED human ID {track_id} (conf={conf:.2f})")

    cv2.imshow("Human Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


✅ SAVED human ID 1 (conf=0.73)
✅ SAVED human ID 2 (conf=0.65)
✅ SAVED human ID 6 (conf=0.76)
✅ SAVED human ID 7 (conf=0.72)
✅ SAVED human ID 9 (conf=0.69)
✅ SAVED human ID 14 (conf=0.65)
✅ SAVED human ID 15 (conf=0.79)
✅ SAVED human ID 18 (conf=0.66)
✅ SAVED human ID 21 (conf=0.69)
✅ SAVED human ID 27 (conf=0.65)


In [1]:
import cv2
import os
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\New folder\video_20260111_144158 - Trim - Trim.mp4"

SAVE_DIR = "detected_humans"
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.65   # confidence threshold

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
saved_track_ids = set()   # save each person only once

print("🚀 Human detection started (terminal-only mode)...")

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        print("📹 Video ended or camera stopped.")
        break

    # YOLO with tracking
    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is None:
        continue

    for box in results.boxes:

        # -------- CLASS FILTER --------
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        # -------- TRACK ID CHECK --------
        if box.id is None:
            continue

        track_id = int(box.id[0])

        # Already saved → skip
        if track_id in saved_track_ids:
            continue

        # -------- CONFIDENCE FILTER --------
        conf = float(box.conf[0])
        if conf < CONF_TH:
            continue

        # -------- DRAW BOUNDING BOX --------
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        save_frame = frame.copy()

        cv2.rectangle(save_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(
            save_frame,
            f"Human ID {track_id} | conf={conf:.2f}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

        # -------- SAVE FRAME --------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(
            SAVE_DIR, f"human_{track_id}_{timestamp}.jpg"
        )

        cv2.imwrite(img_path, save_frame)
        saved_track_ids.add(track_id)

        print(f"✅ SAVED | ID={track_id} | conf={conf:.2f} | file={img_path}")

cap.release()
print("🛑 Detection stopped.")


🚀 Human detection started (terminal-only mode)...
✅ SAVED | ID=1 | conf=0.73 | file=detected_humans\human_1_20260115_143008.jpg
✅ SAVED | ID=2 | conf=0.65 | file=detected_humans\human_2_20260115_143008.jpg
✅ SAVED | ID=6 | conf=0.76 | file=detected_humans\human_6_20260115_143009.jpg
✅ SAVED | ID=7 | conf=0.72 | file=detected_humans\human_7_20260115_143010.jpg
✅ SAVED | ID=9 | conf=0.69 | file=detected_humans\human_9_20260115_143010.jpg
✅ SAVED | ID=14 | conf=0.65 | file=detected_humans\human_14_20260115_143010.jpg
✅ SAVED | ID=15 | conf=0.79 | file=detected_humans\human_15_20260115_143010.jpg
✅ SAVED | ID=18 | conf=0.66 | file=detected_humans\human_18_20260115_143011.jpg
✅ SAVED | ID=21 | conf=0.69 | file=detected_humans\human_21_20260115_143011.jpg
✅ SAVED | ID=27 | conf=0.65 | file=detected_humans\human_27_20260115_143012.jpg
✅ SAVED | ID=30 | conf=0.69 | file=detected_humans\human_30_20260115_143013.jpg
✅ SAVED | ID=31 | conf=0.66 | file=detected_humans\human_31_20260115_143013.jpg


In [2]:
import cv2
import os
import csv
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\New folder\video_20260111_144158 - Trim - Trim.mp4"

SAVE_DIR = "detected_humans"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "detections.csv")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.65

# ===================== DRONE GPS (REPLACE WITH MAVLINK) =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0  # meters

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp",
            "human_id",
            "confidence",
            "drone_lat",
            "drone_lon",
            "drone_alt"
        ])

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
saved_track_ids = set()

print("🚀 Human detection started (GPS logging, no image path)...")

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        print("📹 Video ended or camera stopped.")
        break

    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is None:
        continue

    for box in results.boxes:

        # -------- CLASS FILTER --------
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        # -------- TRACK ID --------
        if box.id is None:
            continue

        track_id = int(box.id[0])

        # Already saved → skip
        if track_id in saved_track_ids:
            continue

        # -------- CONFIDENCE FILTER --------
        conf = float(box.conf[0])
        if conf < CONF_TH:
            continue

        # -------- DRAW BBOX + GPS --------
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        save_frame = frame.copy()

        cv2.rectangle(save_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        overlay_lines = [
            f"Human ID: {track_id}",
            f"Conf: {conf:.2f}",
            f"Lat: {DRONE_LAT:.6f}",
            f"Lon: {DRONE_LON:.6f}",
            f"Alt: {DRONE_ALT:.1f} m"
        ]

        y_text = y1 - 10
        for line in overlay_lines:
            cv2.putText(
                save_frame,
                line,
                (x1, y_text),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.55,
                (0, 255, 0),
                2
            )
            y_text -= 20

        # -------- SAVE IMAGE --------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(
            IMG_DIR, f"human_{track_id}_{timestamp}.jpg"
        )
        cv2.imwrite(img_path, save_frame)

        # -------- SAVE CSV (NO IMAGE PATH) --------
        with open(CSV_PATH, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                timestamp,
                track_id,
                round(conf, 3),
                round(DRONE_LAT, 7),
                round(DRONE_LON, 7),
                DRONE_ALT
            ])

        saved_track_ids.add(track_id)

        print(
            f"✅ SAVED | ID={track_id} | conf={conf:.2f} | "
            f"GPS=({DRONE_LAT:.6f}, {DRONE_LON:.6f}, {DRONE_ALT}m)"
        )

cap.release()
print("🛑 Detection stopped.")


🚀 Human detection started (GPS logging, no image path)...
✅ SAVED | ID=1 | conf=0.73 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=2 | conf=0.65 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=6 | conf=0.76 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=7 | conf=0.72 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=9 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=14 | conf=0.65 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=15 | conf=0.79 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=18 | conf=0.66 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=21 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=27 | conf=0.65 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=30 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=31 | conf=0.66 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=34 | conf=0.71 | GPS=(25.262109, 82.984039, 15.0m)
📹 Video ended or camera stopped.
🛑 Detection stopped.


In [ ]:
import cv2
import os
import csv
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\dowloads\PixVerse_V5.5_Image_Text_360P_I_want_a_video_i (1).mp4"

SAVE_DIR = "detected_humans"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "detections.csv")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.65

# ===================== DRONE GPS (REPLACE WITH MAVLINK) =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0  # meters

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp",
            "human_id",
            "confidence",
            "drone_lat",
            "drone_lon",
            "drone_alt"
        ])

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(0)

# ===================== MEMORY =====================
saved_track_ids = set()
active_person_id = None   # currently handled person

print("🚀 Human detection started (event-based mode)...")

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        print("📹 Video ended.")
        break

    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is None:
        continue

    for box in results.boxes:

        # -------- CLASS FILTER --------
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        if box.id is None:
            continue

        track_id = int(box.id[0])
        conf = float(box.conf[0])

        # Ignore already saved persons forever
        if track_id in saved_track_ids:
            continue

        # If already processing another person → WAIT
        if active_person_id is not None and track_id != active_person_id:
            continue

        if conf < CONF_TH:
            continue

        # -------- NEW PERSON DETECTED --------
        active_person_id = track_id

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        save_frame = frame.copy()

        cv2.rectangle(save_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        overlay = [
            f"Human ID: {track_id}",
            f"Conf: {conf:.2f}",
            f"Lat: {DRONE_LAT:.6f}",
            f"Lon: {DRONE_LON:.6f}",
            f"Alt: {DRONE_ALT:.1f} m"
        ]

        y = y1 - 10
        for line in overlay:
            cv2.putText(
                save_frame,
                line,
                (x1, y),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.55,
                (0, 255, 0),
                2
            )
            y -= 20

        # -------- SAVE IMAGE --------
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img_path = os.path.join(
            IMG_DIR, f"human_{track_id}_{timestamp}.jpg"
        )
        cv2.imwrite(img_path, save_frame)

        # -------- SAVE CSV --------
        with open(CSV_PATH, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                timestamp,
                track_id,
                round(conf, 3),
                round(DRONE_LAT, 7),
                round(DRONE_LON, 7),
                DRONE_ALT
            ])

        saved_track_ids.add(track_id)
        active_person_id = None  # RESET → wait for new person

        print(
            f"✅ SAVED | ID={track_id} | conf={conf:.2f} | "
            f"GPS=({DRONE_LAT:.6f}, {DRONE_LON:.6f}, {DRONE_ALT}m)"
        )

        break  # stop checking other boxes this frame

cap.release()
print("🛑 Detection stopped.")


In [5]:
import cv2
import os
import csv
from datetime import datetime
from ultralytics import YOLO

# ===================== PATHS =====================
MODEL_PATH = r"E:\NIDAR\runs\detect\human_detection_y8n_stage22\weights\best.pt"
VIDEO_PATH = r"F:\dowloads\PixVerse_V5.5_Image_Text_360P_I_want_a_video_i.mp4"

SAVE_DIR = "detected_humans"
IMG_DIR = os.path.join(SAVE_DIR, "images")
CSV_PATH = os.path.join(SAVE_DIR, "detections.csv")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# ===================== PARAMETERS =====================
PERSON_CLASS_ID = 0
CONF_TH = 0.65

# ===================== DRONE GPS (REPLACE WITH MAVLINK) =====================
DRONE_LAT = 25.2621092
DRONE_LON = 82.9840395
DRONE_ALT = 15.0  # meters

# ===================== CSV INIT =====================
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp",
            "human_id",
            "confidence",
            "drone_lat",
            "drone_lon",
            "drone_alt"
        ])

# ===================== MODEL & VIDEO =====================
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

# ===================== MEMORY =====================
saved_track_ids = set()
active_person_id = None

print("🚀 Human detection started (with live output)...")

# ===================== MAIN LOOP =====================
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        print("📹 Video ended.")
        break

    frame_to_show = frame.copy()

    results = model.track(
        frame,
        conf=CONF_TH,
        persist=True,
        verbose=False
    )[0]

    if results.boxes is not None:

        for box in results.boxes:

            # -------- CLASS FILTER --------
            if int(box.cls[0]) != PERSON_CLASS_ID:
                continue

            if box.id is None:
                continue

            track_id = int(box.id[0])
            conf = float(box.conf[0])

            # Ignore already saved persons
            if track_id in saved_track_ids:
                continue

            # Wait until current person is saved
            if active_person_id is not None and track_id != active_person_id:
                continue

            if conf < CONF_TH:
                continue

            active_person_id = track_id

            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # -------- DRAW BBOX --------
            cv2.rectangle(frame_to_show, (x1, y1), (x2, y2), (0, 255, 0), 3)

            overlay = [
                f"Human ID: {track_id}",
                f"Conf: {conf:.2f}",
                f"Lat: {DRONE_LAT:.6f}",
                f"Lon: {DRONE_LON:.6f}",
                f"Alt: {DRONE_ALT:.1f} m"
            ]

            y = y1 - 10
            for line in overlay:
                cv2.putText(
                    frame_to_show,
                    line,
                    (x1, y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.55,
                    (0, 255, 0),
                    2
                )
                y -= 20

            # -------- SAVE IMAGE --------
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            img_path = os.path.join(
                IMG_DIR, f"human_{track_id}_{timestamp}.jpg"
            )
            cv2.imwrite(img_path, frame_to_show)

            # -------- SAVE CSV --------
            with open(CSV_PATH, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    timestamp,
                    track_id,
                    round(conf, 3),
                    round(DRONE_LAT, 7),
                    round(DRONE_LON, 7),
                    DRONE_ALT
                ])

            saved_track_ids.add(track_id)
            active_person_id = None

            print(
                f"✅ SAVED | ID={track_id} | conf={conf:.2f} | "
                f"GPS=({DRONE_LAT:.6f}, {DRONE_LON:.6f}, {DRONE_ALT}m)"
            )

            break  # process only one person per frame

    # ===================== SHOW OUTPUT =====================
    cv2.imshow("Human Detection", frame_to_show)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 User stopped.")
        break

cap.release()
cv2.destroyAllWindows()
print("🛑 Detection stopped.")


🚀 Human detection started (with live output)...
✅ SAVED | ID=1 | conf=0.77 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=4 | conf=0.70 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=5 | conf=0.68 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=6 | conf=0.66 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=7 | conf=0.67 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=9 | conf=0.65 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=10 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=2 | conf=0.74 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=15 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=18 | conf=0.73 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=20 | conf=0.66 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=21 | conf=0.67 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=23 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=24 | conf=0.66 | GPS=(25.262109, 82.984039, 15.0m)
✅ SAVED | ID=27 | conf=0.69 | GPS=(25.262109, 82.984039, 15.0m)